In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
#TODO: uninstall stuff (homebrew etc.)

In [3]:
import os
import numpy as np
import warnings
from astropy.io import fits
from astropy.table import Table
from pyqsofit.PyQSOFit import QSOFit

from main_code.constants import *
from main_code import qsofit
from main_code import data_reading
from main_code import helpers
from main_code import plotting

warnings.simplefilter(action='default') # Or 'always' to see them every time
warnings.showwarning = lambda msg, *args: print(f"WARNING:\n\t{msg}", flush=True)

In [ ]:
# make Ha good first

#TODO: try turning off power law continuum and iron
#TODO: try get v6 back

In [82]:
helpers.get_kwargs_from_log("SDSS_2021_v25", exclude_log_items=True)

{'name': 'SDSS_2021_v25',
 'nsmooth': 1,
 'and_mask': False,
 'or_mask': False,
 'reject_badpix': False,
 'deredden': False,
 'wave_range': (3900, 9000),
 'wave_mask': None,
 'decompose_host': True,
 'host_prior': False,
 'host_prior_scale': 0.2,
 'host_line_mask': True,
 'decomp_na_mask': True,
 'qso_type': 'global',
 'npca_gal': 10,
 'host_type': 'BC03',
 'npca_qso': 5,
 'Fe_uv_op': True,
 'poly': True,
 'BC': False,
 'rej_abs_conti': True,
 'rej_abs_line': True,
 'initial_guess': None,
 'n_pix_min_conti': 50,
 'param_file_name': 'qsopar4.fits',
 'MC': False,
 'MCMC': False,
 'nburn': 20,
 'nsamp': 200,
 'nthin': 10,
 'epsilon_jitter': 0.0,
 'linefit': True,
 'save_result': True,
 'plot_fig': True,
 'save_fig': True,
 'plot_corner': True,
 'save_fits_path': 'output/',
 'save_fits_name': None,
 'verbose': True,
 'kwargs_plot': {'save_fig_path': 'output/', 'broad_fwhm': 1200},
 'kwargs_conti_emcee': {},
 'kwargs_line_emcee': {}}

In [83]:
# Your parameters
use_example_params = False
data_fname = FNAME_2021

lam_bounds = TOTAL_LAM_BOUNDS

params_path = "qsofit_example/" if use_example_params else "data/qsofit/"
params_file_name = "qsopar.fits" if use_example_params else "qsopar4.fits"

output_file_name = helpers.get_output_file_name(fname=data_fname)

filter_bad_values = True
interpolate_bad_values = False

if data_fname in [FNAME_2001, FNAME_2021, FNAME_2022]:
    lam, flux, err = data_reading.get_sdss_lam_flux_err(
        fname=data_fname,
        filter_bad_values=filter_bad_values,
        interpolate_bad_values=interpolate_bad_values,
        lam_bounds=lam_bounds
    )
elif data_fname in [
    FNAME_2015_BLUE_3_ARCSEC, FNAME_2015_RED_3_ARCSEC,
    FNAME_2015_BLUE_4_ARCSEC, FNAME_2015_RED_4_ARCSEC
]:
    lam, flux, err = data_reading.get_sami_lam_flux_err(
        fname=data_fname,
        filter_bad_values=filter_bad_values,
        interpolate_bad_values=interpolate_bad_values,
        lam_bounds=lam_bounds
    )

	divide by zero encountered in divide


In [ ]:
kwargs = {
    "name": output_file_name,                   # default None
    
    "nsmooth": 1,                               # default 1    
    "and_mask": False,                          # default False
    "or_mask": False,                           # default False
    "reject_badpix": False,                     # default True
    "deredden": False,                          # default True
    "wave_range": lam_bounds,                   # default None
    "wave_mask": None,                          # default None
    
    "decompose_host": True,                     # default True
    "host_prior": False,                        # default False
    "host_prior_scale": 0.2,                    # default 0.2
    "host_line_mask": True,                     # default True
    "decomp_na_mask": True,                     # default False
    
    "qso_type": "global",                       # default "global"
    "npca_qso": 10,                             # default 10
    "host_type": "BC03",                        # default "BC03"
    "npca_gal": 5,                              # default 5
    
    "Fe_uv_op": True,                          # default True
    "poly": True,                              # default False
    "BC": False,                                # default False
    "rej_abs_conti": True,                     # default False
    "rej_abs_line": True,                      # default False
    "initial_guess": None,                      # default None
    "n_pix_min_conti": 50,                     # default 100

    "param_file_name": params_file_name,        # default "qsopar.fits"

    "MC": False,                                # default False
    "MCMC": False,                              # default False
    "nburn": 20,                                # default 20
    "nsamp": 200,                               # default 200
    "nthin": 10,                                # default 10
    "epsilon_jitter": 0.,                       # default 0.
    
    "linefit": True,                            # default True
    
    "save_result": True,                        # default True
    "plot_fig": True,                           # default True
    "save_fig": True,                           # default True
    "plot_corner": True,                        # default True
    
    "save_fits_path": "output/",                # default "."
    "save_fits_name": None,                     # default None

    "verbose": False,                            # default False

    "kwargs_plot": {                            # default {}
        "save_fig_path": "output/",
        "broad_fwhm": 1200
    },
    "kwargs_conti_emcee": {},                   # default {}
    "kwargs_line_emcee": {},                    # default {}
}

# ======= Create and run QSOFit =======
q = QSOFit(
    lam=lam, flux=flux, err=err,
    z=Z_SPEC,
    path=params_path
)


q.Fit(**kwargs)


kwargs_log = {
    "output_file_name": output_file_name,
    "data_fname": data_fname,
    "params_file_name": str(params_file_name),
    "lam_bounds": lam_bounds,
    "filter_bad_values": filter_bad_values,
    "interpolate_bad_values": interpolate_bad_values
} | kwargs
kwargs_log.pop("name")
helpers.log_kwargs(kwargs_log)

	unclosed file <_io.BufferedReader name='data/qsofit/qsopar4.fits'>
	Mean of empty slice.
	invalid value encountered in scalar divide
	Degrees of freedom <= 0 for slice
	invalid value encountered in divide
	invalid value encountered in scalar divide
	Mean of empty slice.
	unclosed file <_io.BufferedReader name='/opt/anaconda3/envs/denison/lib/python3.11/site-packages/pyqsofit/pca/Yip_pca_templates/qso_eigenspec_Yip2004_global.fits'>
	Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
	unclosed file <_io.BufferedReader name='/opt/anaconda3/envs/denison/lib/python3.11/site-packages/pyqsofit/indo/G8III__141680.fits'>
	unclosed file <_io.BufferedReader name='/opt/anaconda3/envs/denison/lib/python3.11/site-packages/pyqsofit/indo/F8V__G_74-5.fits'>
	unclosed file <_io.BufferedReader name='/opt/anaconda3/envs/denison/lib/python3.11/site-packages

In [58]:
qsofit.edit_qsopar(
    value_name_to_edit="inisca",
    new_value=1,
    col_name_and_val=("linename", "NII6549"),
    file_name="qsopar14.fits",
    make_copy=False
)

inisca for linename = NII6549 updated successfully from [0.001] to 1.
Changes saved to original file: data/qsofit/qsopar14.fits


In [ ]:
# Read the line priors
qsopar = fits.open('qsofit_example/qsopar.fits')
line_priors = Table(qsopar['LINE_PRIORS'].data)

# Display the Hα and Hβ complex lines
print(line_priors[['linename', 'ngauss', 'minsig', 'maxsig', 'vindex', 'windex']])

$$M_{BH} = (2.0^{+0.4}_{-0.3}) \cdot 10^6 \cdot \left[\frac{L_{H\alpha}}{10^{42} \text{ erg s}^{-1}}\right]^{0.55 \pm 0.02} \cdot \left[\frac{FWHM_{H\alpha}}{10^3 \text{ km s}^{-1}}\right]^{2.06 \pm 0.06} M_\odot$$

$$\log M = \log a + b \cdot \log L + c \cdot \log V$$

$$\delta(\log M) = \delta(\log a) + \log L \cdot \delta b + b \cdot \delta(\log L) + \log V \cdot \delta c + c \cdot \delta(\log V)$$

$$\sigma^2_{\log M} = \sigma^2_{\log a} + (\log L)^2 \sigma^2_b + b^2 \sigma^2_{\log L} + (\log V)^2 \sigma^2_c + c^2 \sigma^2_{\log V}$$

$$\sigma_{\log_{10} x} = \frac{1}{\ln 10} \cdot \frac{\delta x}{x}$$

$$\delta M = \ln(10) \cdot M \cdot \sigma_{\log_{10} M}$$

 if $y = \log_{10}(x)$, then $\delta x = \ln(10) \cdot x \cdot \delta y$.

In [ ]:
import numpy as np
import scipy.optimize as spo
import scipy.signal as sps
import matplotlib.pyplot as plt

from main_code import helpers

# --- Step 1: Generate the data ---
# Your original data (a noisy inverted parabola)
np.random.seed(42)
x = np.linspace(-100, 100, 500)
y_true = (
    2000 * np.exp(-(x + 40)**2 / (2 * 15**2)) +  # Peak 1
    3500 * np.exp(-(x - 10)**2 / (2 * 25**2)) +  # Peak 2
    1500 * np.exp(-(x - 60)**2 / (2 * 10**2))    # Peak 3
)
noise = np.random.normal(0, 200, len(x))
y = y_true + noise

# --- Step 2: Define the Gaussian model function ---
# A Gaussian has 3 parameters: amplitude (A), center (mu), and width (sigma)
# The functional form is: f(x) = A * exp(-(x - mu)^2 / (2 * sigma^2))
def my_gaussian(x, A, mu, sigma):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2))


def n_gaussians(x, n, peaks, mus, sigmas):
    """Sum of n Gaussians"""
    res = np.zeros_like(x)
    for i in range(n):
        res += peaks[i] * np.exp(-(x - mus[i])**2 / (2 * sigmas[i]**2))
    return res

def get_initial_guesses(
    n: int,
    x: np.ndarray,
    y: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    peak = np.ones(n) * np.max(y)
    mu_guesses = np.ones(n) * x[np.argmax(y)]

    peak_indices = [np.argmax(y)] * n
    fwhm_guesses, _, _, _ = sps.peak_widths(y, peak_indices, rel_height=0.5) # returns width in indices at half max
    peak_x_diffs = helpers.get_masked_diffs(x, peak_indices)
    fwhm_guesses = fwhm_guesses * peak_x_diffs
    sigma_guesses = fwhm_guesses / SIGMA_TO_FWHM

def multi_gaussian(x, A1, mu1, sigma1, A2, mu2, sigma2):
    """Sum of two Gaussians"""
    g1 = A1 * np.exp(-(x - mu1)**2 / (2 * sigma1**2))
    g2 = A2 * np.exp(-(x - mu2)**2 / (2 * sigma2**2))
    return g1 + g2

# Initial guesses for 2 peaks (you'd adjust these based on your data)
p0_multi = [3000, -30, 30,   # First Gaussian: A1, mu1, sigma1
            3000,  30, 30]   # Second Gaussian: A2, mu2, sigma2

popt_multi, pcov_multi = spo.curve_fit(multi_gaussian, x, y, p0=p0_multi)

perr_multi = np.sqrt(np.diag(pcov_multi))

# --- Step 3: Provide initial guesses for the parameters ---
# Good initial guesses help the optimizer converge faster and avoid local minima
# - A (amplitude): Use the maximum of the data
# - mu (center): Use the x-value where y is maximum
# - sigma (width): Estimate from the data spread (e.g., std dev of x where y > half-max)
A_guess = np.max(y)                    # Peak height
mu_guess = x[np.argmax(y)]             # x-position of peak
sigma_guess = 50                        # Rough estimate of width

p0 = [A_guess, mu_guess, sigma_guess]

# --- Step 4: Perform the curve fit ---
# curve_fit uses non-linear least squares (Levenberg-Marquardt by default)
# It minimizes the sum of squared residuals: Σ(y_data - y_model)^2
popt, pcov = spo.curve_fit(my_gaussian, x, y, p0=p0)

# popt: optimal parameters [A, mu, sigma]
# pcov: covariance matrix (diagonal gives variance of each parameter)

# --- Step 5: Extract results and uncertainties ---
# A_fit, mu_fit, sigma_fit = popt
A_fit_1, mu_fit_1, sigma_fit_1, A_fit_2, mu_fit_2, sigma_fit_2 = popt_multi
# perr = np.sqrt(np.diag(pcov))  # Standard errors from covariance matrix
perr = np.sqrt(np.diag(pcov_multi))  # Standard errors from covariance matrix

print(f"Fitted parameters:")
print(f"  Amplitude A = {A_fit_1:.2f} ± {perr[0]:.2f}")
print(f"  Center μ    = {mu_fit_1:.2f} ± {perr[1]:.2f}")
print(f"  Width σ     = {sigma_fit_1:.2f} ± {perr[2]:.2f}")

# --- Step 6: Plot the result ---
plt.figure(figsize=(10, 6))
plt.scatter(x, y, s=10, alpha=0.5, label='Data')
plt.plot(x, my_gaussian(x, *popt_multi[0:3]), 'r-', linewidth=2, label='Gaussian fit')
plt.plot(x, my_gaussian(x, *popt_multi[3:6]), 'r-', linewidth=2, label='Gaussian fit')
plt.plot(x, my_gaussian(x, *popt_multi[0:3]) + my_gaussian(x, *popt_multi[3:6]), 'r-', linewidth=2, label='Total fit')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Single Gaussian Fit')
plt.show()